In [7]:
from __future__ import print_function
import os, math, random, pickle, time
import numpy as np
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler,EarlyStopping
from keras.models import Sequential, Model, load_model
from keras.layers.pooling import AveragePooling1D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation, Input, merge, Convolution2D, Reshape, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
import keras
from sklearn import preprocessing
from keras import backend as K
from keras import metrics
import matplotlib.pyplot as plt

def get_score(y_true, y_pred):
    s=0
    for i in range(len(y_true)):
        d = y_pred[i] - y_true[i]
        if d < 0:
            s+=math.e**(-d/13)-1
        else:
            s+=math.e**(d/10)-1
    return s

def step_decay(epoch):
    lrat = 0
    if epoch<200:
        lrat = 0.001
    else:
        lrat = 0.0001
    return lrat  

FeatureN = 14
nb_epoch = 250
batch_size = 512
FilterN = 10
FilterL = 10
rmse,sco,tm = [], [], []


#writer = open('DCNN_5C_30TW_noRearly.pkl', 'wb')
ConstRUL = 125
TW = 30
Dataset = '1'
# Time Window Max of Dataset
#1:31, 2:21, 3:38, 4:19
#1:125, 2: 155, 3:125， 4: 155

############ training samples ##################################

setTrain = {'1':100, '2':260, '3':100, '4':248}
setTest = {'1':100, '2':259, '3':100, '4':248}
nTrain = setTrain[Dataset]
nTest = setTest[Dataset]

data = [] 
for line in open("../CMAPSSData/train_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
data = min_max_scaler.fit_transform(data)
num=[]
for i in range(nTrain):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    num.append(tmp)
num=np.array(num)

label=[]
for i in range(nTrain):
    label.append([])
    length = len(num[i])
    for j in range(length):
        label[i].append(ConstRUL if length-j-1>=ConstRUL else length-j-1)
label = np.array(label)

samples,targets,noofsample = [],[],[]
for i in range(nTrain):
    noofsample.append(len(num[i])-TW+1)
    for j in range(noofsample[-1]):
        samples.append(num[i][j:j+TW,:])
        targets.append(label[i][j+TW-1])
samples = np.array(samples)
targets = np.array(targets)

################## testing data ###########################
data = [] 
for line in open("../CMAPSSData/test_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
data = min_max_scaler.transform(data)
numt=[]
for i in range(nTest):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    numt.append(tmp)
numt=np.array(numt)

samplet, count_miss = [],[]
for i in range(nTest):
    if len(numt[i])>=TW:
        samplet.append(numt[i][-TW:,:])
    else:
        count_miss.append(i)
samplet = np.array(samplet)

labelt = [] 
for line in open("../CMAPSSData/RUL_FD00"+Dataset+".txt"):
    labelt.append(line.split())
labelt = np.cast['int32'](labelt)
labelnew = []
for i in range(nTest):
    if i not in count_miss:
        #labelnew.append(labelt[i][0])
        labelnew.append(labelt[i][0] if labelt[i][0]<=ConstRUL else ConstRUL)
labelt = labelnew
labelt=np.array(labelt)

seed = 2222
np.random.seed(seed)
np.random.shuffle(samples)
np.random.seed(seed)
np.random.shuffle(targets)
samplet = samplet[np.argsort(labelt)]
labelt = labelt[np.argsort(labelt)]

print(samples.shape)

###########################################################

for i in range(1):
    print(i)
    start = time.clock()
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C1')(y)
    y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C2')(y)
    y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C3')(y)
    y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Convolution2D(1, 3, 1, border_mode='same', init='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = keras.layers.Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', init='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    opt = keras.optimizers.Adam(lr=0, beta_1=0.5)
    DCNN = Model([input_layer], [y])
    #DCNN.compile(loss=get_score,optimizer=opt)
    DCNN.compile(loss='mean_squared_error',optimizer=opt)
    lrate = LearningRateScheduler(step_decay)
    history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
                     validation_data=(samplet, labelt), callbacks=[lrate])
    
    #, TensorBoard(log_dir='tmp\\tan_4c_4')
    
    #history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
    #                 validation_data=(samplet, labelt), callbacks=[lrate])
    #history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=0, callbacks=[lrate])
    
    score = DCNN.evaluate(samplet, labelt, batch_size=batch_size, verbose=1)
    print('Test score:', score)
    end = time.clock()
    
    rmse.append(np.sqrt(score))
    sco.append(get_score(labelt, DCNN.predict(samplet)))
    tm.append(end-start)
    
    #DCNN.save('DCNN_5C_noR_'+str(i)+'.h5')
    #DCNN.save_weights('DCNN_5C_'+str(i))

rec = np.array([[np.average(rmse), np.std(rmse)],[np.average(sco), np.std(sco)],[np.average(tm), np.std(tm)]])
print(rec)
#pickle.dump(rec, writer)

#np.savetxt('file.txt', rec, delimiter=' ')
#DCNN.save_weights('DCNN_5C')

#writer.close()

#writer = open('Dataset1_30TW_data.pkl', 'wb')
#pickle.dump([samplet, labelt], writer)
#writer.close()

(17731, 30, 14)
0


C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:141: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), activation="tanh", kernel_initializer="glorot_normal", padding="same", name="C1")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:142: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), activation="tanh", kernel_initializer="glorot_normal", padding="same", name="C2")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:143: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), activation="tanh", kernel_initializer="glorot_normal", padding="same", name="C3")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:144: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), activation="tanh", kernel_initializer="glorot_normal", padding=

Train on 17731 samples, validate on 100 samples
Epoch 1/250
17731/17731 [==============================] - 1s 32us/step - loss: 6554.7945 - val_loss: 4958.7324
Epoch 2/250
17731/17731 [==============================] - 0s 22us/step - loss: 5620.1881 - val_loss: 4534.6870
Epoch 3/250
17731/17731 [==============================] - 0s 22us/step - loss: 5173.8787 - val_loss: 4162.1118
Epoch 4/250
17731/17731 [==============================] - 0s 22us/step - loss: 4773.8147 - val_loss: 3824.4128
Epoch 5/250
17731/17731 [==============================] - 0s 22us/step - loss: 4408.0566 - val_loss: 3512.8965
Epoch 6/250
17731/17731 [==============================] - 0s 22us/step - loss: 4071.1164 - val_loss: 3226.0068
Epoch 7/250
17731/17731 [==============================] - 0s 22us/step - loss: 3760.8594 - val_loss: 2963.7866
Epoch 8/250
17731/17731 [==============================] - 0s 21us/step - loss: 3475.1612 - val_loss: 2725.5068
Epoch 9/250
17731/17731 [==============================]

17731/17731 [==============================] - 0s 22us/step - loss: 209.3307 - val_loss: 179.4967
Epoch 75/250
17731/17731 [==============================] - 0s 22us/step - loss: 216.9465 - val_loss: 228.7159
Epoch 76/250
17731/17731 [==============================] - 0s 22us/step - loss: 195.4520 - val_loss: 156.1523
Epoch 77/250
17731/17731 [==============================] - 0s 22us/step - loss: 211.4894 - val_loss: 183.9736
Epoch 78/250
17731/17731 [==============================] - 0s 21us/step - loss: 192.5434 - val_loss: 198.6824
Epoch 79/250
17731/17731 [==============================] - 0s 22us/step - loss: 219.0437 - val_loss: 178.3968
Epoch 80/250
17731/17731 [==============================] - 0s 22us/step - loss: 196.9481 - val_loss: 188.5345
Epoch 81/250
17731/17731 [==============================] - 0s 21us/step - loss: 198.5363 - val_loss: 223.9199
Epoch 82/250
17731/17731 [==============================] - 0s 22us/step - loss: 200.3998 - val_loss: 201.8222
Epoch 83/250
1

17731/17731 [==============================] - 0s 22us/step - loss: 157.9374 - val_loss: 195.8936
Epoch 148/250
17731/17731 [==============================] - 0s 22us/step - loss: 153.7742 - val_loss: 184.8046
Epoch 149/250
17731/17731 [==============================] - 0s 22us/step - loss: 165.8624 - val_loss: 180.5034
Epoch 150/250
17731/17731 [==============================] - 0s 22us/step - loss: 152.8582 - val_loss: 157.4161
Epoch 151/250
17731/17731 [==============================] - 0s 22us/step - loss: 160.3057 - val_loss: 160.8432
Epoch 152/250
17731/17731 [==============================] - 0s 22us/step - loss: 156.7853 - val_loss: 163.6556
Epoch 153/250
17731/17731 [==============================] - 0s 22us/step - loss: 152.3672 - val_loss: 157.5548
Epoch 154/250
17731/17731 [==============================] - 0s 22us/step - loss: 172.7531 - val_loss: 169.2614
Epoch 155/250
17731/17731 [==============================] - 0s 22us/step - loss: 154.1786 - val_loss: 164.5968
Epoch 

17731/17731 [==============================] - 0s 22us/step - loss: 133.2712 - val_loss: 169.6172
Epoch 221/250
17731/17731 [==============================] - 0s 21us/step - loss: 132.7458 - val_loss: 167.6307
Epoch 222/250
17731/17731 [==============================] - 0s 22us/step - loss: 134.0879 - val_loss: 169.8834
Epoch 223/250
17731/17731 [==============================] - 0s 22us/step - loss: 132.6803 - val_loss: 176.2135
Epoch 224/250
17731/17731 [==============================] - 0s 21us/step - loss: 132.8354 - val_loss: 170.5703
Epoch 225/250
17731/17731 [==============================] - 0s 22us/step - loss: 133.3752 - val_loss: 174.2134
Epoch 226/250
17731/17731 [==============================] - 0s 22us/step - loss: 134.4966 - val_loss: 170.9823
Epoch 227/250
17731/17731 [==============================] - 0s 22us/step - loss: 132.1325 - val_loss: 182.3448
Epoch 228/250
17731/17731 [==============================] - 0s 22us/step - loss: 132.2701 - val_loss: 174.4261
Epoch 

In [ ]:
def plotRUL(cycles, rulArray, nnPred, cnnPred, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.plot(cycles, cnnPred, 'ro-', label='CNN Pred')
    plt.legend()
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    plt.show()

In [5]:
data = [] 
for line in open("../CMAPSSData/test_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
data = min_max_scaler.fit_transform(data)
num=[]
for i in range(nTrain):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    num.append(tmp)
num=np.array(num)

samples,targets,noofsample = [],[],[]
for i in range(nTrain):
    noofsample.append(len(num[i])-TW+1)
    for j in range(noofsample[-1]):
        samples.append(num[i][j:j+TW,:])
samples = np.array(samples)

In [6]:
engineUnit = 21

print(samples.shape)


'''cnnPred = DCNN.predict(samplet)

maxCycle = X_test2.shape[0]
faultCycle = y_test[engineUnit-1]
cycles = np.arange(maxCycle)
rulArray = np.arange(faultCycle, maxCycle+faultCycle)
rulArray[rulArray > constRUL] = constRUL
rulArray = np.flipud(rulArray)'''

#print(cycles)
#print(rulArray)

'''print("Testing data")
print(X_test2.shape)
print(X_test2[-5:,:])
print(nnPred)
print(cnnPred)'''

#plotRUL(cycles, rulArray, nnPred, cnnPred, engineUnit)

(10196, 30, 14)


'print("Testing data")\nprint(X_test2.shape)\nprint(X_test2[-5:,:])\nprint(nnPred)\nprint(cnnPred)'